### Recommending a game to a user

In [8]:
import mysql.connector 

connection = mysql.connector.connect(
    host = "localhost",
    user = "root",
    passwd = "password",
    database = "steam_data"
)


In [9]:
sql = "SELECT COUNT(*) FROM steam_purchase;"
print(sql)

SELECT COUNT(*) FROM steam_play;


In [10]:
cursor = connection.cursor()
cursor.execute(sql)
result = cursor.fetchall()
print(result)
cursor.close()

[(70488,)]


In [16]:
from graphtastic.database.mysql import query_mysql

The steam_purchase table contains information on user purchases. 

In [14]:
result = query_mysql("SELECT * FROM steam_purchase LIMIT 5;", password=PASSWORD)
print(result)

[('151603712', 'Fallout 4', 'purchase', 1.0, 0), ('151603712', 'Spore', 'purchase', 1.0, 0), ('151603712', 'Fallout New Vegas', 'purchase', 1.0, 0), ('151603712', 'Left 4 Dead 2', 'purchase', 1.0, 0), ('151603712', 'HuniePop', 'purchase', 1.0, 0), ('151603712', 'Path of Exile', 'purchase', 1.0, 0), ('151603712', 'Poly Bridge', 'purchase', 1.0, 0), ('151603712', 'Left 4 Dead', 'purchase', 1.0, 0), ('151603712', 'Team Fortress 2', 'purchase', 1.0, 0), ('151603712', 'Tomb Raider', 'purchase', 1.0, 0)]


In [15]:
for row in result:
    print(row)

('151603712', 'Fallout 4', 'purchase', 1.0, 0)
('151603712', 'Spore', 'purchase', 1.0, 0)
('151603712', 'Fallout New Vegas', 'purchase', 1.0, 0)
('151603712', 'Left 4 Dead 2', 'purchase', 1.0, 0)
('151603712', 'HuniePop', 'purchase', 1.0, 0)
('151603712', 'Path of Exile', 'purchase', 1.0, 0)
('151603712', 'Poly Bridge', 'purchase', 1.0, 0)
('151603712', 'Left 4 Dead', 'purchase', 1.0, 0)
('151603712', 'Team Fortress 2', 'purchase', 1.0, 0)
('151603712', 'Tomb Raider', 'purchase', 1.0, 0)


In the steam_price table, the first column again contains a unique user ID. As with steam_play, the second column contains a game's name. The third column always contains the purchase string. The fourth column in this table always contains floatt 1.0, indicating the purchase of that row's game, by the user.

At a glance, we can see that there is data for user 151603712 in both the steam_play
and steam_purchase tables. So, users in our datasets have data about both the purchase
of games and play time. From a brief look into the data, we can see that there are plenty of
columns that we won’t need going forward, so we can start to exclude them from our analysis
by querying specific rows.

### Path-based analytics in tabular data

Now that we have an idea of what our tables contain, we can ask some questions about our data. For
example, we can ask, **“What other games may a user enjoy, based on their previous purchase habits?”**
This type of question forms the basis of a recommendation system, used across the sales and marketing
industry to drive additional customer purchases.

To answer a question like this simply, we will need to find the games a user has purchased, then find other users that have purchased the same games. Then, we will look at commonalities between those users to recommend a new game to our original user.


First, we will need to query MySQL to find all games purchased by a specific user. Let’s use the
user at the top of the steam_purchase table as an example, 151603712.

In [17]:
result = query_mysql("SELECT game_name FROM steam_purchase WHERE id = '151603712';", password="Met")
print(result[:10])
print(len(result))

[('Fallout 4',), ('Spore',), ('Fallout New Vegas',), ('Left 4 Dead 2',), ('HuniePop',), ('Path of Exile',), ('Poly Bridge',), ('Left 4 Dead',), ('Team Fortress 2',), ('Tomb Raider',)]
39


From the print() statements, we can see that this user has purchased 40 separate games. Note
that MySQL has returned the data as a list of tuples, where each tuple has an empty element.
We will need to access the first element of each tuple and enclose each game in quote marks
to find the users associated with these games.

In [18]:
games = ['"' + game[0] + '"' for game in result]
games_string = ','.join(games)
print(games_string)

"Fallout 4","Spore","Fallout New Vegas","Left 4 Dead 2","HuniePop","Path of Exile","Poly Bridge","Left 4 Dead","Team Fortress 2","Tomb Raider","The Banner Saga","Dead Island Epidemic","BioShock Infinite","Dragon Age Origins - Ultimate Edition","Fallout 3 - Game of the Year Edition","SEGA Genesis & Mega Drive Classics","Grand Theft Auto IV","Realm of the Mad God","Marvel Heroes 2015","Eldevin","Dota 2","BioShock","Robocraft","Garry's Mod","Jazzpunk","Alan Wake","BioShock 2","Fallen Earth","Fallout New Vegas Courier's Stash","Fallout New Vegas Dead Money","Fallout New Vegas Honest Hearts","Grand Theft Auto Episodes from Liberty City","Hitman Absolution","HuniePop Official Digital Art Collection","HuniePop Original Soundtrack","The Banner Saga - Mod Content","The Elder Scrolls V Skyrim - Dawnguard","The Elder Scrolls V Skyrim - Dragonborn","The Elder Scrolls V Skyrim - Hearthfire"


Now, we can query the table again with these results to find the users that also purchased these games.

In [29]:
query_2 = f"SELECT id FROM steam_purchase WHERE game_name" \
    f" IN ({games_string}) AND id != '151603712';"
    
result_2 = query_mysql(query_2, password="Met")
users = [user[0] for user in result_2]
print(users[:10])
print(len(users))

['187131847', '59945701', '59945701', '59945701', '59945701', '59945701', '59945701', '59945701', '53875128', '53875128']
15507


We have a list of 15507 users in our result, but this contains duplicates, so let's remove them with a set()

In [30]:
users = list(set(users))
print(users[:10])
print(len(users))

['163043651', '244491524', '95325409', '47978182', '255010251', '153148942', '118447144', '176306969', '30280104', '184554757']
7660


Now, we need to find new games to recommend to our selected user based on the purchases of our identified users. Let's set up the parameters for a new SQL query based on these requirements

In [31]:
users = ['"' + user + '"' for user in users]

users_string = ','.join(users)

query_3 = f'SELECT game_name from steam_purchase WHERE id IN ({users_string}) ' \
      	f'AND game_name NOT IN ({games_string});'
 
result_3 = query_mysql(query_3, password="Met")
recommended_games = [game[0] for game in result_3]
print(recommended_games[:10])

['Ultra Street Fighter IV', 'FINAL FANTASY XIII', 'The Elder Scrolls V Skyrim', "Sid Meier's Civilization V", 'L.A. Noire', 'Company of Heroes Tales of Valor', '7 Days to Die', 'Divekick', 'FINAL FANTASY VII', 'Orcs Must Die! 2']


In [32]:
from collections import  Counter
game_frequency = Counter(recommended_games)
print(game_frequency)

Counter({'Unturned': 1118, 'Counter-Strike Global Offensive': 1094, 'Warframe': 729, 'The Elder Scrolls V Skyrim': 648, 'Counter-Strike Source': 588, 'Heroes & Generals': 538, 'Half-Life 2 Lost Coast': 529, 'Portal 2': 504, 'War Thunder': 500, 'Portal': 485, 'Terraria': 434, 'Half-Life 2': 430, 'Counter-Strike': 427, 'Borderlands 2': 413, 'PAYDAY 2': 401, "Sid Meier's Civilization V": 400, 'Half-Life 2 Deathmatch': 393, 'No More Room in Hell': 366, 'Counter-Strike Condition Zero': 364, 'Counter-Strike Condition Zero Deleted Scenes': 364, 'PlanetSide 2': 346, 'Grand Theft Auto San Andreas': 329, 'Half-Life 2 Episode One': 320, 'Half-Life 2 Episode Two': 317, 'Nosgoth': 317, 'Trove': 314, 'Loadout': 310, 'Counter-Strike Nexon Zombies': 292, 'Metro 2033': 284, 'Dirty Bomb': 279, "Tom Clancy's Ghost Recon Phantoms - NA": 276, 'PAYDAY The Heist': 267, 'Warface': 267, 'Neverwinter': 257, 'The Witcher 2 Assassins of Kings Enhanced Edition': 250, 'Saints Row The Third': 249, 'Arma 2': 245, 'Al

This dictionary of game frequencies tells us that we could recommend the game Unturned to our specified user.

Taking data models into account, think about the operations involved in finding the game recommendation. Using a relational database, we had to repeatedly query the same table with increasingly large results. This could also have been achieved with several self-joins of the `steam_purchase` table. 